In [1]:
globals().clear
import time
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()
from datetime import datetime

In [2]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler

import tensorflow       as tf
import keras.optimizers as op
from keras.callbacks import EarlyStopping

from tensorflow                     import keras
from tensorflow.keras.models        import Sequential
from tensorflow.keras.layers        import Dense
from tensorflow.keras.layers        import Dropout
from tensorflow.keras.optimizers    import Adam

In [3]:
# Load Dataset
df = pd.read_csv('../df.csv')

In [4]:
# list of tickers for stocks in our data set. Sort the ticker list since our df will be alphabetically arranged.
tickers = ["AAPL", 'XOM', 'IBM', 'KO', 'CVX', 'BA', 'PFE', 'MSFT', 'T', 'WMT',
       'F', 'NFLX', 'JPM', 'MCD', 'GE', 'NVDA', 'JNJ', 'BAC', 'C', 'AMZN',
       'INTC', 'CSCO', 'TSLA', 'GOOGL', 'AMD', 'BABA', 'VZ', 'DIS',
       'META', 'PLTR']
tickers.sort()

In [5]:
# For this specific trial drop PLTR since data is not complete
df = df[df.stock_ID != "PLTR"]

In [6]:
df['DATETIME']= pd.to_datetime(df['DATETIME'], format='%m/%d/%Y %H:%M')

In [7]:
# Sort by time so that the first 29 rows occupy the first time value for all the stocks. 
# Fill in 0's for missing values for now.
df = df.set_index('DATETIME')
df = df.fillna(0)

In [8]:
# Select the length of the df ; For this file we need 7 months
# We use the first 3 months to make the first prediction, then shift window
# Then we will repeat this for the next 4 months (Hence total is 7 months needed)
df = df.sort_index().loc['2022-06-01':'2022-12-30']

In [9]:
#Set the DATETIME for fecha 
DATETIME = df.index.values[::29]

In [10]:
# Sort dataframe such that it is both in sequential order, and also in alphabetical order for each day 
#(i.e first entry for each time entry should be AAPL, and last should be XOM).
df.sort_values(["DATETIME", "stock_ID"], inplace=True)

In [11]:
# Select individual stocks and remove demographics
aapl = df.iloc[::29  ,:74]
f    = df.iloc[10::29,:74]
nvda = df.iloc[22::29,:74]
tsla = df.iloc[25::29,:74]
wmt  = df.iloc[27::29,:74]

In [12]:
num_companies = 1  # 29 companies in our dataset.
step_rows  = 24 * num_companies  # 24 time periods per day per stock
total_rows = len(aapl['2022-06-01':'2022-12-30'])  # Define total length to predict on
train_rows = len(aapl['2022-06-01':'2022-08-31'])  # Define length of training window


scale_X = MinMaxScaler()
model_1 = Sequential()
model_1.add(Dense(100, activation = 'relu', input_dim = aapl.iloc[:,1:-1].shape[1]))  # initiating with 100 neurons
model_1.add(Dense(100, activation = 'relu', input_dim = aapl.iloc[:,1:-1].shape[1]))
model_1.add(Dropout(0.25))                                                          # adding dropout to avoid overfitting
model_1.add(Dense(1))                                                               # output layer

opt = Adam(amsgrad = True, lr = 0.001, beta_1 = 0.79, beta_2 = 0.999)             # using Adam optimizer, at a learning rate of 0.001
model_1.compile(loss = 'mse', optimizer = opt)                                      # compiling model
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=4)

aapl_result = pd.DataFrame(columns=['DATETIME', 'ACTUAL', 'PREDICTED', 'DIFFERENCE', 'TRAIN_DURATION'])

for i in range(0, total_rows - train_rows, step_rows):
    st = time.time()
    # 1. Obtain X and y
    train  = aapl.iloc[i:train_rows + i, 1:] 
    test   = aapl.iloc[train_rows + i:train_rows + i + step_rows, 1:]  
    X_train, y_train = train.iloc[:, 1:], train.iloc[:, 0]
    X_test, y_test = test.iloc[:, 1:], test.iloc[:, 0]


    
    # 2. Scale X
    X_train = scale_X.fit_transform(X_train)
    X_test = scale_X.transform(X_test)

    # 3. Fit and Predict
    model_1.fit(X_train, y_train, epochs=200, batch_size=256, callbacks=[es])
    y_hat = model_1.predict(X_test, verbose=False)
    et = time.time()

    # 4. Save data with prediction
    fecha = DATETIME[train_rows + i:train_rows + i + step_rows]
    datos = {
        'DATETIME': fecha.ravel(),
        'ACTUAL': y_test.ravel(),
        'PREDICTED': y_hat.ravel(),
        'DIFFERENCE': abs(y_hat.ravel() - y_test.ravel()),
        'TRAIN_DURATION': np.full(y_hat.ravel().shape[0], et - st)
    }
    data = pd.DataFrame(data=datos)
    aapl_result = pd.concat([aapl_result, data], ignore_index=True)

    print("Count Down:", int((total_rows - train_rows - i) / step_rows))
    # print(data.head())

Epoch 1/200


C:\Users\Gurbir\anaconda3\envs\tf\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


6/6 [==============================] - 1s 4ms/step - loss: 23326.0469
Epoch 2/200
6/6 [==============================] - 0s 4ms/step - loss: 23051.7754
Epoch 3/200
6/6 [==============================] - 0s 4ms/step - loss: 22639.7090
Epoch 4/200
6/6 [==============================] - 0s 4ms/step - loss: 21982.1465
Epoch 5/200
6/6 [==============================] - 0s 4ms/step - loss: 20931.5039
Epoch 6/200
6/6 [==============================] - 0s 4ms/step - loss: 19343.8066
Epoch 7/200
6/6 [==============================] - 0s 4ms/step - loss: 17126.8008
Epoch 8/200
6/6 [==============================] - 0s 4ms/step - loss: 14317.1523
Epoch 9/200
6/6 [==============================] - 0s 5ms/step - loss: 10950.3359
Epoch 10/200
6/6 [==============================] - 0s 4ms/step - loss: 7541.5430
Epoch 11/200
6/6 [==============================] - 0s 4ms/step - loss: 4441.6841
Epoch 12/200
6/6 [==============================] - 0s 4ms/step - loss: 2159.6431
Epoch 13/200
6/6 [==========

Epoch 2/200
6/6 [==============================] - 0s 4ms/step - loss: 150.4100
Epoch 3/200
6/6 [==============================] - 0s 4ms/step - loss: 153.7436
Epoch 4/200
6/6 [==============================] - 0s 4ms/step - loss: 161.5059
Epoch 5/200
6/6 [==============================] - 0s 4ms/step - loss: 156.0664
Epoch 6/200
6/6 [==============================] - 0s 4ms/step - loss: 165.0958
Epoch 6: early stopping
Count Down: 73
Epoch 1/200
6/6 [==============================] - 0s 4ms/step - loss: 156.2204
Epoch 2/200
6/6 [==============================] - 0s 4ms/step - loss: 151.4993
Epoch 3/200
6/6 [==============================] - 0s 4ms/step - loss: 155.9672
Epoch 4/200
6/6 [==============================] - 0s 4ms/step - loss: 157.0989
Epoch 5/200
6/6 [==============================] - 0s 5ms/step - loss: 153.9135
Epoch 6/200
6/6 [==============================] - 0s 4ms/step - loss: 155.1337
Epoch 6: early stopping
Count Down: 72
Epoch 1/200
6/6 [=========================

6/6 [==============================] - 0s 4ms/step - loss: 144.8261
Epoch 3/200
6/6 [==============================] - 0s 4ms/step - loss: 150.3594
Epoch 4/200
6/6 [==============================] - 0s 3ms/step - loss: 145.8210
Epoch 5/200
6/6 [==============================] - 0s 5ms/step - loss: 154.0991
Epoch 6/200
6/6 [==============================] - 0s 4ms/step - loss: 151.5317
Epoch 6: early stopping
Count Down: 49
Epoch 1/200
6/6 [==============================] - 0s 4ms/step - loss: 149.7957
Epoch 2/200
6/6 [==============================] - 0s 4ms/step - loss: 150.6700
Epoch 3/200
6/6 [==============================] - 0s 4ms/step - loss: 154.4335
Epoch 4/200
6/6 [==============================] - 0s 4ms/step - loss: 161.7042
Epoch 5/200
6/6 [==============================] - 0s 4ms/step - loss: 152.2387
Epoch 5: early stopping
Count Down: 48
Epoch 1/200
6/6 [==============================] - 0s 4ms/step - loss: 151.3238
Epoch 2/200
6/6 [==============================] - 0s 

6/6 [==============================] - 0s 4ms/step - loss: 141.3199
Epoch 10/200
6/6 [==============================] - 0s 4ms/step - loss: 140.7739
Epoch 11/200
6/6 [==============================] - 0s 4ms/step - loss: 138.9843
Epoch 12/200
6/6 [==============================] - 0s 3ms/step - loss: 140.9227
Epoch 12: early stopping
Count Down: 25
Epoch 1/200
6/6 [==============================] - 0s 4ms/step - loss: 140.9841
Epoch 2/200
6/6 [==============================] - 0s 4ms/step - loss: 148.5631
Epoch 3/200
6/6 [==============================] - 0s 4ms/step - loss: 135.8636
Epoch 4/200
6/6 [==============================] - 0s 4ms/step - loss: 142.8426
Epoch 5/200
6/6 [==============================] - 0s 3ms/step - loss: 147.3839
Epoch 6/200
6/6 [==============================] - 0s 4ms/step - loss: 134.7280
Epoch 7/200
6/6 [==============================] - 0s 4ms/step - loss: 140.4957
Epoch 8/200
6/6 [==============================] - 0s 4ms/step - loss: 143.4593
Epoch 9/2

6/6 [==============================] - 0s 4ms/step - loss: 119.3732
Epoch 3/200
6/6 [==============================] - 0s 4ms/step - loss: 128.4386
Epoch 4/200
6/6 [==============================] - 0s 4ms/step - loss: 127.9146
Epoch 5/200
6/6 [==============================] - 0s 3ms/step - loss: 133.9250
Epoch 6/200
6/6 [==============================] - 0s 4ms/step - loss: 123.7372
Epoch 6: early stopping
Count Down: 4
Epoch 1/200
6/6 [==============================] - 0s 3ms/step - loss: 131.5055
Epoch 2/200
6/6 [==============================] - 0s 4ms/step - loss: 125.0476
Epoch 3/200
6/6 [==============================] - 0s 3ms/step - loss: 123.3308
Epoch 4/200
6/6 [==============================] - 0s 4ms/step - loss: 126.7899
Epoch 5/200
6/6 [==============================] - 0s 4ms/step - loss: 122.4402
Epoch 6/200
6/6 [==============================] - 0s 4ms/step - loss: 127.1253
Epoch 7/200
6/6 [==============================] - 0s 4ms/step - loss: 125.2469
Epoch 8/200
6/

In [13]:
num_companies = 1  # 29 companies in our dataset.
step_rows  = 24 * num_companies  # 24 time periods per day per stock
total_rows = len(f['2022-06-01':'2022-12-30'])  # Define total length to predict on
train_rows = len(f['2022-06-01':'2022-08-31'])  # Define length of training window


scale_X = MinMaxScaler()
model_2 = Sequential()
model_2.add(Dense(100, activation = 'relu', input_dim = f.iloc[:,1:-1].shape[1]))  # initiating with 100 neurons
model_2.add(Dense(100, activation = 'relu', input_dim = f.iloc[:,1:-1].shape[1]))
model_2.add(Dropout(0.25))                                                          # adding dropout to avoid overfitting
model_2.add(Dense(1))                                                               # output layer

opt = Adam(amsgrad = True, lr = 0.001, beta_1 = 0.79, beta_2 = 0.999)             # using Adam optimizer, at a learning rate of 0.001
model_2.compile(loss = 'mse', optimizer = opt)                                      # compiling model
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=4)

f_result = pd.DataFrame(columns=['DATETIME', 'ACTUAL', 'PREDICTED', 'DIFFERENCE', 'TRAIN_DURATION'])

for i in range(0, total_rows - train_rows, step_rows):
    st = time.time()
    # 1. Obtain X and y
    train  = f.iloc[i:train_rows + i, 1:] 
    test   = f.iloc[train_rows + i:train_rows + i + step_rows, 1:]  
    X_train, y_train = train.iloc[:, 1:], train.iloc[:, 0]
    X_test, y_test = test.iloc[:, 1:], test.iloc[:, 0]


    
    # 2. Scale X
    X_train = scale_X.fit_transform(X_train)
    X_test = scale_X.transform(X_test)

    # 3. Fit and Predict
    model_2.fit(X_train, y_train, epochs=200, batch_size=256, callbacks=[es])
    y_hat = model_2.predict(X_test, verbose=False)
    et = time.time()

    # 4. Save data with prediction
    fecha = DATETIME[train_rows + i:train_rows + i + step_rows]
    datos = {
        'DATETIME': fecha.ravel(),
        'ACTUAL': y_test.ravel(),
        'PREDICTED': y_hat.ravel(),
        'DIFFERENCE': abs(y_hat.ravel() - y_test.ravel()),
        'TRAIN_DURATION': np.full(y_hat.ravel().shape[0], et - st)
    }
    data = pd.DataFrame(data=datos)
    f_result = pd.concat([f_result, data], ignore_index=True)

    print("Count Down:", int((total_rows - train_rows - i) / step_rows))
    # print(data.head())

Epoch 1/200


C:\Users\Gurbir\anaconda3\envs\tf\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


6/6 [==============================] - 0s 3ms/step - loss: 175.3692
Epoch 2/200
6/6 [==============================] - 0s 4ms/step - loss: 153.5984
Epoch 3/200
6/6 [==============================] - 0s 4ms/step - loss: 127.4902
Epoch 4/200
6/6 [==============================] - 0s 4ms/step - loss: 93.0038
Epoch 5/200
6/6 [==============================] - 0s 4ms/step - loss: 52.5118
Epoch 6/200
6/6 [==============================] - 0s 4ms/step - loss: 17.9367
Epoch 7/200
6/6 [==============================] - 0s 4ms/step - loss: 4.2705
Epoch 8/200
6/6 [==============================] - 0s 5ms/step - loss: 3.7635
Epoch 9/200
6/6 [==============================] - 0s 4ms/step - loss: 3.0039
Epoch 10/200
6/6 [==============================] - 0s 4ms/step - loss: 2.5573
Epoch 11/200
6/6 [==============================] - 0s 5ms/step - loss: 2.4948
Epoch 12/200
6/6 [==============================] - 0s 4ms/step - loss: 2.2016
Epoch 13/200
6/6 [==============================] - 0s 4ms/step 

6/6 [==============================] - 0s 4ms/step - loss: 1.4708
Epoch 4/200
6/6 [==============================] - 0s 4ms/step - loss: 1.4984
Epoch 5/200
6/6 [==============================] - 0s 3ms/step - loss: 1.4125
Epoch 6/200
6/6 [==============================] - 0s 4ms/step - loss: 1.4807
Epoch 6: early stopping
Count Down: 61
Epoch 1/200
6/6 [==============================] - 0s 3ms/step - loss: 1.5158
Epoch 2/200
6/6 [==============================] - 0s 4ms/step - loss: 1.5836
Epoch 3/200
6/6 [==============================] - 0s 4ms/step - loss: 1.4417
Epoch 4/200
6/6 [==============================] - 0s 4ms/step - loss: 1.4194
Epoch 5/200
6/6 [==============================] - 0s 4ms/step - loss: 1.5047
Epoch 6/200
6/6 [==============================] - 0s 4ms/step - loss: 1.4477
Epoch 7/200
6/6 [==============================] - 0s 4ms/step - loss: 1.4784
Epoch 8/200
6/6 [==============================] - 0s 4ms/step - loss: 1.4942
Epoch 8: early stopping
Count Down: 6

6/6 [==============================] - 0s 4ms/step - loss: 1.3247
Epoch 3/200
6/6 [==============================] - 0s 4ms/step - loss: 1.4149
Epoch 4/200
6/6 [==============================] - 0s 4ms/step - loss: 1.3816
Epoch 5/200
6/6 [==============================] - 0s 4ms/step - loss: 1.3896
Epoch 6/200
6/6 [==============================] - 0s 3ms/step - loss: 1.3548
Epoch 6: early stopping
Count Down: 38
Epoch 1/200
6/6 [==============================] - 0s 4ms/step - loss: 1.3721
Epoch 2/200
6/6 [==============================] - 0s 4ms/step - loss: 1.4516
Epoch 3/200
6/6 [==============================] - 0s 4ms/step - loss: 1.2604
Epoch 4/200
6/6 [==============================] - 0s 4ms/step - loss: 1.4011
Epoch 5/200
6/6 [==============================] - 0s 4ms/step - loss: 1.3264
Epoch 6/200
6/6 [==============================] - 0s 4ms/step - loss: 1.3121
Epoch 7/200
6/6 [==============================] - 0s 4ms/step - loss: 1.3888
Epoch 7: early stopping
Count Down: 3

6/6 [==============================] - 0s 5ms/step - loss: 1.3108
Epoch 5/200
6/6 [==============================] - 0s 4ms/step - loss: 1.2183
Epoch 6/200
6/6 [==============================] - 0s 4ms/step - loss: 1.3450
Epoch 7/200
6/6 [==============================] - 0s 4ms/step - loss: 1.1891
Epoch 8/200
6/6 [==============================] - 0s 4ms/step - loss: 1.3352
Epoch 9/200
6/6 [==============================] - 0s 5ms/step - loss: 1.2839
Epoch 10/200
6/6 [==============================] - 0s 5ms/step - loss: 1.3503
Epoch 11/200
6/6 [==============================] - 0s 5ms/step - loss: 1.2961
Epoch 11: early stopping
Count Down: 26
Epoch 1/200
6/6 [==============================] - 0s 5ms/step - loss: 1.2210
Epoch 2/200
6/6 [==============================] - 0s 4ms/step - loss: 1.3679
Epoch 3/200
6/6 [==============================] - 0s 4ms/step - loss: 1.3084
Epoch 4/200
6/6 [==============================] - 0s 4ms/step - loss: 1.2541
Epoch 5/200
6/6 [=================

Epoch 5/200
6/6 [==============================] - 0s 4ms/step - loss: 1.2214
Epoch 6/200
6/6 [==============================] - 0s 5ms/step - loss: 1.2096
Epoch 7/200
6/6 [==============================] - 0s 5ms/step - loss: 1.2239
Epoch 8/200
6/6 [==============================] - 0s 5ms/step - loss: 1.2538
Epoch 9/200
6/6 [==============================] - 0s 5ms/step - loss: 1.2380
Epoch 10/200
6/6 [==============================] - 0s 5ms/step - loss: 1.2186
Epoch 10: early stopping
Count Down: 14
Epoch 1/200
6/6 [==============================] - 0s 4ms/step - loss: 1.2064
Epoch 2/200
6/6 [==============================] - 0s 5ms/step - loss: 1.1445
Epoch 3/200
6/6 [==============================] - 0s 4ms/step - loss: 1.2680
Epoch 4/200
6/6 [==============================] - 0s 5ms/step - loss: 1.2177
Epoch 5/200
6/6 [==============================] - 0s 5ms/step - loss: 1.2165
Epoch 6/200
6/6 [==============================] - 0s 5ms/step - loss: 1.2005
Epoch 6: early stopping

In [14]:
num_companies = 1  # 29 companies in our dataset.
step_rows  = 24 * num_companies  # 24 time periods per day per stock
total_rows = len(nvda['2022-06-01':'2022-12-30'])  # Define total length to predict on
train_rows = len(nvda['2022-06-01':'2022-08-31'])  # Define length of training window


scale_X = MinMaxScaler()
model_3 = Sequential()
model_3.add(Dense(100, activation = 'relu', input_dim = nvda.iloc[:,1:-1].shape[1]))  # initiating with 100 neurons
model_3.add(Dense(100, activation = 'relu', input_dim = nvda.iloc[:,1:-1].shape[1]))
model_3.add(Dropout(0.25))                                                          # adding dropout to avoid overfitting
model_3.add(Dense(1))                                                               # output layer

opt = Adam(amsgrad = True, lr = 0.001, beta_1 = 0.79, beta_2 = 0.999)             # using Adam optimizer, at a learning rate of 0.001
model_3.compile(loss = 'mse', optimizer = opt)                                      # compiling model
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=4)

nvda_result = pd.DataFrame(columns=['DATETIME', 'ACTUAL', 'PREDICTED', 'DIFFERENCE', 'TRAIN_DURATION'])

for i in range(0, total_rows - train_rows, step_rows):
    st = time.time()
    # 1. Obtain X and y
    train  = nvda.iloc[i:train_rows + i, 1:] 
    test   = nvda.iloc[train_rows + i:train_rows + i + step_rows, 1:]  
    X_train, y_train = train.iloc[:, 1:], train.iloc[:, 0]
    X_test, y_test = test.iloc[:, 1:], test.iloc[:, 0]


    
    # 2. Scale X
    X_train = scale_X.fit_transform(X_train)
    X_test = scale_X.transform(X_test)

    # 3. Fit and Predict
    model_3.fit(X_train, y_train, epochs=200, batch_size=256, callbacks=[es])
    y_hat = model_3.predict(X_test, verbose=False)
    et = time.time()

    # 4. Save data with prediction
    fecha = DATETIME[train_rows + i:train_rows + i + step_rows]
    datos = {
        'DATETIME': fecha.ravel(),
        'ACTUAL': y_test.ravel(),
        'PREDICTED': y_hat.ravel(),
        'DIFFERENCE': abs(y_hat.ravel() - y_test.ravel()),
        'TRAIN_DURATION': np.full(y_hat.ravel().shape[0], et - st)
    }
    data = pd.DataFrame(data=datos)
    nvda_result = pd.concat([nvda_result, data], ignore_index=True)

    print("Count Down:", int((total_rows - train_rows - i) / step_rows))
    # print(data.head())

Epoch 1/200


C:\Users\Gurbir\anaconda3\envs\tf\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


6/6 [==============================] - 0s 4ms/step - loss: 29127.2891
Epoch 2/200
6/6 [==============================] - 0s 4ms/step - loss: 28848.4453
Epoch 3/200
6/6 [==============================] - 0s 5ms/step - loss: 28455.8027
Epoch 4/200
6/6 [==============================] - 0s 5ms/step - loss: 27817.2031
Epoch 5/200
6/6 [==============================] - 0s 4ms/step - loss: 26769.7559
Epoch 6/200
6/6 [==============================] - 0s 4ms/step - loss: 25121.4922
Epoch 7/200
6/6 [==============================] - 0s 5ms/step - loss: 22769.5059
Epoch 8/200
6/6 [==============================] - 0s 5ms/step - loss: 19591.9980
Epoch 9/200
6/6 [==============================] - 0s 5ms/step - loss: 15761.5117
Epoch 10/200
6/6 [==============================] - 0s 4ms/step - loss: 11475.7959
Epoch 11/200
6/6 [==============================] - 0s 5ms/step - loss: 7271.4985
Epoch 12/200
6/6 [==============================] - 0s 5ms/step - loss: 3872.9309
Epoch 13/200
6/6 [=========

Epoch 10/200
6/6 [==============================] - 0s 4ms/step - loss: 151.1105
Epoch 11/200
6/6 [==============================] - 0s 5ms/step - loss: 154.2398
Epoch 11: early stopping
Count Down: 64
Epoch 1/200
6/6 [==============================] - 0s 4ms/step - loss: 153.2023
Epoch 2/200
6/6 [==============================] - 0s 4ms/step - loss: 151.4356
Epoch 3/200
6/6 [==============================] - 0s 4ms/step - loss: 156.2178
Epoch 4/200
6/6 [==============================] - 0s 4ms/step - loss: 146.4493
Epoch 5/200
6/6 [==============================] - 0s 4ms/step - loss: 151.3847
Epoch 6/200
6/6 [==============================] - 0s 4ms/step - loss: 151.4241
Epoch 7/200
6/6 [==============================] - 0s 5ms/step - loss: 147.0004
Epoch 8/200
6/6 [==============================] - 0s 4ms/step - loss: 151.5918
Epoch 8: early stopping
Count Down: 63
Epoch 1/200
6/6 [==============================] - 0s 4ms/step - loss: 148.8501
Epoch 2/200
6/6 [======================

6/6 [==============================] - 0s 5ms/step - loss: 121.5940
Epoch 8/200
6/6 [==============================] - 0s 4ms/step - loss: 126.8930
Epoch 8: early stopping
Count Down: 40
Epoch 1/200
6/6 [==============================] - 0s 5ms/step - loss: 126.6686
Epoch 2/200
6/6 [==============================] - 0s 4ms/step - loss: 119.7083
Epoch 3/200
6/6 [==============================] - 0s 4ms/step - loss: 124.3653
Epoch 4/200
6/6 [==============================] - 0s 4ms/step - loss: 120.8534
Epoch 5/200
6/6 [==============================] - 0s 4ms/step - loss: 122.1873
Epoch 6/200
6/6 [==============================] - 0s 5ms/step - loss: 125.5883
Epoch 6: early stopping
Count Down: 39
Epoch 1/200
6/6 [==============================] - 0s 5ms/step - loss: 123.5510
Epoch 2/200
6/6 [==============================] - 0s 5ms/step - loss: 130.9470
Epoch 3/200
6/6 [==============================] - 0s 4ms/step - loss: 123.0029
Epoch 4/200
6/6 [==============================] - 0s 

Epoch 1/200
6/6 [==============================] - 0s 5ms/step - loss: 110.1721
Epoch 2/200
6/6 [==============================] - 0s 5ms/step - loss: 109.9544
Epoch 3/200
6/6 [==============================] - 0s 4ms/step - loss: 110.4366
Epoch 4/200
6/6 [==============================] - 0s 4ms/step - loss: 114.1722
Epoch 5/200
6/6 [==============================] - 0s 4ms/step - loss: 116.4939
Epoch 6/200
6/6 [==============================] - 0s 5ms/step - loss: 114.1725
Epoch 6: early stopping
Count Down: 29
Epoch 1/200
6/6 [==============================] - 0s 5ms/step - loss: 110.2691
Epoch 2/200
6/6 [==============================] - 0s 5ms/step - loss: 106.6375
Epoch 3/200
6/6 [==============================] - 0s 4ms/step - loss: 109.9071
Epoch 4/200
6/6 [==============================] - 0s 4ms/step - loss: 117.9928
Epoch 5/200
6/6 [==============================] - 0s 4ms/step - loss: 115.2916
Epoch 6/200
6/6 [==============================] - 0s 8ms/step - loss: 121.9920
E

Epoch 11: early stopping
Count Down: 18
Epoch 1/200
6/6 [==============================] - 0s 5ms/step - loss: 107.7171
Epoch 2/200
6/6 [==============================] - 0s 5ms/step - loss: 112.9202
Epoch 3/200
6/6 [==============================] - 0s 5ms/step - loss: 110.1712
Epoch 4/200
6/6 [==============================] - 0s 5ms/step - loss: 104.9332
Epoch 5/200
6/6 [==============================] - 0s 5ms/step - loss: 105.1537
Epoch 6/200
6/6 [==============================] - 0s 5ms/step - loss: 115.5702
Epoch 7/200
6/6 [==============================] - 0s 5ms/step - loss: 109.8185
Epoch 8/200
6/6 [==============================] - 0s 5ms/step - loss: 110.9833
Epoch 8: early stopping
Count Down: 17
Epoch 1/200
6/6 [==============================] - 0s 5ms/step - loss: 101.5763
Epoch 2/200
6/6 [==============================] - 0s 4ms/step - loss: 104.8002
Epoch 3/200
6/6 [==============================] - 0s 4ms/step - loss: 109.6609
Epoch 4/200
6/6 [========================

In [15]:
num_companies = 1  # 29 companies in our dataset.
step_rows  = 24 * num_companies  # 24 time periods per day per stock
total_rows = len(tsla['2022-06-01':'2022-12-30'])  # Define total length to predict on
train_rows = len(tsla['2022-06-01':'2022-08-31'])  # Define length of training window


scale_X = MinMaxScaler()
model_4 = Sequential()
model_4.add(Dense(100, activation = 'relu', input_dim = tsla.iloc[:,1:-1].shape[1]))  # initiating with 100 neurons
model_4.add(Dense(100, activation = 'relu', input_dim = tsla.iloc[:,1:-1].shape[1]))
model_4.add(Dropout(0.25))                                                          # adding dropout to avoid overfitting
model_4.add(Dense(1))                                                               # output layer

opt = Adam(amsgrad = True, lr = 0.001, beta_1 = 0.79, beta_2 = 0.999)             # using Adam optimizer, at a learning rate of 0.001
model_4.compile(loss = 'mse', optimizer = opt)                                      # compiling model
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=4)

tsla_result = pd.DataFrame(columns=['DATETIME', 'ACTUAL', 'PREDICTED', 'DIFFERENCE', 'TRAIN_DURATION'])

for i in range(0, total_rows - train_rows, step_rows):
    st = time.time()
    # 1. Obtain X and y
    train  = tsla.iloc[i:train_rows + i, 1:] 
    test   = tsla.iloc[train_rows + i:train_rows + i + step_rows, 1:]  
    X_train, y_train = train.iloc[:, 1:], train.iloc[:, 0]
    X_test, y_test = test.iloc[:, 1:], test.iloc[:, 0]


    
    # 2. Scale X
    X_train = scale_X.fit_transform(X_train)
    X_test = scale_X.transform(X_test)

    # 3. Fit and Predict
    model_4.fit(X_train, y_train, epochs=200, batch_size=256, callbacks=[es])
    y_hat = model_4.predict(X_test, verbose=False)
    et = time.time()

    # 4. Save data with prediction
    fecha = DATETIME[train_rows + i:train_rows + i + step_rows]
    datos = {
        'DATETIME': fecha.ravel(),
        'ACTUAL': y_test.ravel(),
        'PREDICTED': y_hat.ravel(),
        'DIFFERENCE': abs(y_hat.ravel() - y_test.ravel()),
        'TRAIN_DURATION': np.full(y_hat.ravel().shape[0], et - st)
    }
    data = pd.DataFrame(data=datos)
    tsla_result = pd.concat([tsla_result, data], ignore_index=True)

    print("Count Down:", int((total_rows - train_rows - i) / step_rows))
    # print(data.head())

Epoch 1/200


C:\Users\Gurbir\anaconda3\envs\tf\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


6/6 [==============================] - 0s 4ms/step - loss: 69311.6328
Epoch 2/200
6/6 [==============================] - 0s 4ms/step - loss: 68896.0234
Epoch 3/200
6/6 [==============================] - 0s 5ms/step - loss: 68407.1328
Epoch 4/200
6/6 [==============================] - 0s 4ms/step - loss: 67712.1250
Epoch 5/200
6/6 [==============================] - 0s 5ms/step - loss: 66657.9062
Epoch 6/200
6/6 [==============================] - 0s 5ms/step - loss: 65041.4414
Epoch 7/200
6/6 [==============================] - 0s 5ms/step - loss: 62578.1445
Epoch 8/200
6/6 [==============================] - 0s 4ms/step - loss: 59089.7031
Epoch 9/200
6/6 [==============================] - 0s 4ms/step - loss: 54426.0781
Epoch 10/200
6/6 [==============================] - 0s 4ms/step - loss: 48538.8789
Epoch 11/200
6/6 [==============================] - 0s 4ms/step - loss: 41604.5117
Epoch 12/200
6/6 [==============================] - 0s 4ms/step - loss: 33896.9258
Epoch 13/200
6/6 [=======

6/6 [==============================] - 0s 4ms/step - loss: 702.4285
Epoch 5/200
6/6 [==============================] - 0s 4ms/step - loss: 665.9828
Epoch 6/200
6/6 [==============================] - 0s 4ms/step - loss: 696.6567
Epoch 7/200
6/6 [==============================] - 0s 4ms/step - loss: 701.1009
Epoch 8/200
6/6 [==============================] - 0s 5ms/step - loss: 695.0185
Epoch 9/200
6/6 [==============================] - 0s 4ms/step - loss: 707.1740
Epoch 9: early stopping
Count Down: 66
Epoch 1/200
6/6 [==============================] - 0s 5ms/step - loss: 679.2183
Epoch 2/200
6/6 [==============================] - 0s 4ms/step - loss: 703.4397
Epoch 3/200
6/6 [==============================] - 0s 4ms/step - loss: 708.6747
Epoch 4/200
6/6 [==============================] - 0s 4ms/step - loss: 682.2400
Epoch 5/200
6/6 [==============================] - 0s 4ms/step - loss: 736.5535
Epoch 5: early stopping
Count Down: 65
Epoch 1/200
6/6 [==============================] - 0s 

Epoch 4/200
6/6 [==============================] - 0s 5ms/step - loss: 664.6833
Epoch 5/200
6/6 [==============================] - 0s 4ms/step - loss: 632.4221
Epoch 6/200
6/6 [==============================] - 0s 4ms/step - loss: 701.8610
Epoch 7/200
6/6 [==============================] - 0s 5ms/step - loss: 722.0441
Epoch 8/200
6/6 [==============================] - 0s 5ms/step - loss: 660.7377
Epoch 9/200
6/6 [==============================] - 0s 4ms/step - loss: 643.3198
Epoch 9: early stopping
Count Down: 44
Epoch 1/200
6/6 [==============================] - 0s 5ms/step - loss: 646.3176
Epoch 2/200
6/6 [==============================] - 0s 4ms/step - loss: 687.8571
Epoch 3/200
6/6 [==============================] - 0s 4ms/step - loss: 698.9075
Epoch 4/200
6/6 [==============================] - 0s 4ms/step - loss: 634.8497
Epoch 5/200
6/6 [==============================] - 0s 4ms/step - loss: 656.2255
Epoch 6/200
6/6 [==============================] - 0s 5ms/step - loss: 689.5881
E

6/6 [==============================] - 0s 5ms/step - loss: 519.6301
Epoch 7/200
6/6 [==============================] - 0s 5ms/step - loss: 517.7265
Epoch 8/200
6/6 [==============================] - 0s 4ms/step - loss: 498.5299
Epoch 9/200
6/6 [==============================] - 0s 5ms/step - loss: 499.5808
Epoch 9: early stopping
Count Down: 22
Epoch 1/200
6/6 [==============================] - 0s 5ms/step - loss: 461.1139
Epoch 2/200
6/6 [==============================] - 0s 4ms/step - loss: 476.6403
Epoch 3/200
6/6 [==============================] - 0s 6ms/step - loss: 459.2940
Epoch 4/200
6/6 [==============================] - 0s 5ms/step - loss: 524.5823
Epoch 5/200
6/6 [==============================] - 0s 4ms/step - loss: 498.8826
Epoch 6/200
6/6 [==============================] - 0s 5ms/step - loss: 458.0138
Epoch 7/200
6/6 [==============================] - 0s 5ms/step - loss: 499.6071
Epoch 8/200
6/6 [==============================] - 0s 4ms/step - loss: 492.6394
Epoch 9/200
6

Epoch 2/200
6/6 [==============================] - 0s 4ms/step - loss: 416.2707
Epoch 3/200
6/6 [==============================] - 0s 4ms/step - loss: 433.2856
Epoch 4/200
6/6 [==============================] - 0s 5ms/step - loss: 444.0195
Epoch 5/200
6/6 [==============================] - 0s 4ms/step - loss: 470.9621
Epoch 6/200
6/6 [==============================] - 0s 4ms/step - loss: 443.1920
Epoch 6: early stopping
Count Down: 11
Epoch 1/200
6/6 [==============================] - 0s 5ms/step - loss: 443.7688
Epoch 2/200
6/6 [==============================] - 0s 4ms/step - loss: 392.0805
Epoch 3/200
6/6 [==============================] - 0s 4ms/step - loss: 416.5246
Epoch 4/200
6/6 [==============================] - 0s 5ms/step - loss: 436.9132
Epoch 5/200
6/6 [==============================] - 0s 5ms/step - loss: 438.3509
Epoch 6/200
6/6 [==============================] - 0s 4ms/step - loss: 395.2174
Epoch 6: early stopping
Count Down: 10
Epoch 1/200
6/6 [=========================

In [16]:
num_companies = 1  # 29 companies in our dataset.
step_rows  = 24 * num_companies  # 24 time periods per day per stock
total_rows = len(wmt['2022-06-01':'2022-12-30'])  # Define total length to predict on
train_rows = len(wmt['2022-06-01':'2022-08-31'])  # Define length of training window


scale_X = MinMaxScaler()
model_5 = Sequential()
model_5.add(Dense(100, activation = 'relu', input_dim = wmt.iloc[:,1:-1].shape[1]))  # initiating with 100 neurons
model_5.add(Dense(100, activation = 'relu', input_dim = wmt.iloc[:,1:-1].shape[1]))
model_5.add(Dropout(0.25))                                                          # adding dropout to avoid overfitting
model_5.add(Dense(1))                                                               # output layer

opt = Adam(amsgrad = True, lr = 0.001, beta_1 = 0.79, beta_2 = 0.999)             # using Adam optimizer, at a learning rate of 0.001
model_5.compile(loss = 'mse', optimizer = opt)                                      # compiling model
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=4)

wmt_result = pd.DataFrame(columns=['DATETIME', 'ACTUAL', 'PREDICTED', 'DIFFERENCE', 'TRAIN_DURATION'])

for i in range(0, total_rows - train_rows, step_rows):
    st = time.time()
    # 1. Obtain X and y
    train  = wmt.iloc[i:train_rows + i, 1:] 
    test   = wmt.iloc[train_rows + i:train_rows + i + step_rows, 1:]  
    X_train, y_train = train.iloc[:, 1:], train.iloc[:, 0]
    X_test, y_test = test.iloc[:, 1:], test.iloc[:, 0]


    
    # 2. Scale X
    X_train = scale_X.fit_transform(X_train)
    X_test = scale_X.transform(X_test)

    # 3. Fit and Predict
    model_5.fit(X_train, y_train, epochs=200, batch_size=256, callbacks=[es])
    y_hat = model_5.predict(X_test, verbose=False)
    et = time.time()

    # 4. Save data with prediction
    fecha = DATETIME[train_rows + i:train_rows + i + step_rows]
    datos = {
        'DATETIME': fecha.ravel(),
        'ACTUAL': y_test.ravel(),
        'PREDICTED': y_hat.ravel(),
        'DIFFERENCE': abs(y_hat.ravel() - y_test.ravel()),
        'TRAIN_DURATION': np.full(y_hat.ravel().shape[0], et - st)
    }
    data = pd.DataFrame(data=datos)
    wmt_result = pd.concat([wmt_result, data], ignore_index=True)

    print("Count Down:", int((total_rows - train_rows - i) / step_rows))
    # print(data.head())

Epoch 1/200


C:\Users\Gurbir\anaconda3\envs\tf\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


6/6 [==============================] - 0s 4ms/step - loss: 16221.2393
Epoch 2/200
6/6 [==============================] - 0s 5ms/step - loss: 16007.8330
Epoch 3/200
6/6 [==============================] - 0s 5ms/step - loss: 15674.5264
Epoch 4/200
6/6 [==============================] - 0s 5ms/step - loss: 15127.5078
Epoch 5/200
6/6 [==============================] - 0s 5ms/step - loss: 14269.9561
Epoch 6/200
6/6 [==============================] - 0s 5ms/step - loss: 12987.0781
Epoch 7/200
6/6 [==============================] - 0s 5ms/step - loss: 11252.5400
Epoch 8/200
6/6 [==============================] - 0s 4ms/step - loss: 9038.8418
Epoch 9/200
6/6 [==============================] - 0s 5ms/step - loss: 6612.9863
Epoch 10/200
6/6 [==============================] - 0s 5ms/step - loss: 4201.6968
Epoch 11/200
6/6 [==============================] - 0s 5ms/step - loss: 2185.8318
Epoch 12/200
6/6 [==============================] - 0s 5ms/step - loss: 896.2964
Epoch 13/200
6/6 [=============

Epoch 4/200
6/6 [==============================] - 0s 4ms/step - loss: 98.2645
Epoch 5/200
6/6 [==============================] - 0s 4ms/step - loss: 104.1885
Epoch 6/200
6/6 [==============================] - 0s 4ms/step - loss: 105.1920
Epoch 7/200
6/6 [==============================] - 0s 4ms/step - loss: 108.5957
Epoch 8/200
6/6 [==============================] - 0s 4ms/step - loss: 100.4519
Epoch 8: early stopping
Count Down: 64
Epoch 1/200
6/6 [==============================] - 0s 4ms/step - loss: 101.5711
Epoch 2/200
6/6 [==============================] - 0s 4ms/step - loss: 100.5295
Epoch 3/200
6/6 [==============================] - 0s 4ms/step - loss: 98.9830
Epoch 4/200
6/6 [==============================] - 0s 4ms/step - loss: 100.9343
Epoch 5/200
6/6 [==============================] - 0s 4ms/step - loss: 102.8701
Epoch 6/200
6/6 [==============================] - 0s 4ms/step - loss: 99.5255
Epoch 7/200
6/6 [==============================] - 0s 4ms/step - loss: 111.6129
Epoc

6/6 [==============================] - 0s 5ms/step - loss: 101.1977
Epoch 6/200
6/6 [==============================] - 0s 4ms/step - loss: 96.0338
Epoch 7/200
6/6 [==============================] - 0s 5ms/step - loss: 109.1833
Epoch 8/200
6/6 [==============================] - 0s 5ms/step - loss: 98.0987
Epoch 9/200
6/6 [==============================] - 0s 5ms/step - loss: 106.5739
Epoch 10/200
6/6 [==============================] - 0s 4ms/step - loss: 102.8713
Epoch 10: early stopping
Count Down: 40
Epoch 1/200
6/6 [==============================] - 0s 4ms/step - loss: 106.0682
Epoch 2/200
6/6 [==============================] - 0s 5ms/step - loss: 109.0927
Epoch 3/200
6/6 [==============================] - 0s 4ms/step - loss: 100.4369
Epoch 4/200
6/6 [==============================] - 0s 4ms/step - loss: 106.9841
Epoch 5/200
6/6 [==============================] - 0s 5ms/step - loss: 101.6703
Epoch 6/200
6/6 [==============================] - 0s 5ms/step - loss: 110.0403
Epoch 7/200
6

6/6 [==============================] - 0s 4ms/step - loss: 104.4002
Epoch 9/200
6/6 [==============================] - 0s 4ms/step - loss: 109.9975
Epoch 10/200
6/6 [==============================] - 0s 5ms/step - loss: 110.9198
Epoch 11/200
6/6 [==============================] - 0s 4ms/step - loss: 114.5030
Epoch 12/200
6/6 [==============================] - 0s 4ms/step - loss: 106.2608
Epoch 12: early stopping
Count Down: 29
Epoch 1/200
6/6 [==============================] - 0s 4ms/step - loss: 110.6785
Epoch 2/200
6/6 [==============================] - 0s 5ms/step - loss: 108.2077
Epoch 3/200
6/6 [==============================] - 0s 4ms/step - loss: 105.9539
Epoch 4/200
6/6 [==============================] - 0s 4ms/step - loss: 116.5717
Epoch 5/200
6/6 [==============================] - 0s 5ms/step - loss: 110.6242
Epoch 6/200
6/6 [==============================] - 0s 4ms/step - loss: 110.9329
Epoch 7/200
6/6 [==============================] - 0s 6ms/step - loss: 107.3093
Epoch 7: 

6/6 [==============================] - 0s 5ms/step - loss: 116.3635
Epoch 12: early stopping
Count Down: 6
Epoch 1/200
6/6 [==============================] - 0s 4ms/step - loss: 130.6010
Epoch 2/200
6/6 [==============================] - 0s 4ms/step - loss: 124.0550
Epoch 3/200
6/6 [==============================] - 0s 5ms/step - loss: 121.6253
Epoch 4/200
6/6 [==============================] - 0s 4ms/step - loss: 117.1467
Epoch 5/200
6/6 [==============================] - 0s 4ms/step - loss: 112.4538
Epoch 6/200
6/6 [==============================] - 0s 4ms/step - loss: 120.4558
Epoch 7/200
6/6 [==============================] - 0s 5ms/step - loss: 119.9511
Epoch 8/200
6/6 [==============================] - 0s 5ms/step - loss: 121.1275
Epoch 9/200
6/6 [==============================] - 0s 4ms/step - loss: 123.2996
Epoch 9: early stopping
Count Down: 5
Epoch 1/200
6/6 [==============================] - 0s 5ms/step - loss: 121.8812
Epoch 2/200
6/6 [==============================] - 0s 4

In [17]:
metrics_df = pd.DataFrame(columns = ['stock_ID', "RMSE", "MAPE", "MPE", "MTT"])

In [18]:
def mean_positive_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    error = np.mean(np.maximum((y_pred - y_true),0))
    return error

In [19]:
new_rows = []
stock_result = aapl_result
# Calculate RMSE 
rmse = np.sqrt(mean_squared_error(stock_result['ACTUAL'], stock_result['PREDICTED']))
# Calculate MAPE % 
mape = mean_absolute_percentage_error(stock_result['ACTUAL'], stock_result['PREDICTED']) * 100
# Calculate MPE % 
mpe = mean_positive_error(stock_result['ACTUAL'],stock_result['PREDICTED'])
# Calculate MTT in seconds 
mtt = np.mean(stock_result['TRAIN_DURATION'])
new_row = pd.Series(['AAPL',rmse, mape, mpe, mtt], index=metrics_df.columns)
new_rows.append(new_row)
metrics_df = pd.DataFrame(new_rows)
metrics_df

,stock_ID,RMSE,MAPE,MPE,MTT
0,AAPL,6.561078,3.302506,2.255839,0.361799


In [20]:
new_rows = []
stock_result = f_result
# Calculate RMSE 
rmse = np.sqrt(mean_squared_error(stock_result['ACTUAL'], stock_result['PREDICTED']))
# Calculate MAPE % 
mape = mean_absolute_percentage_error(stock_result['ACTUAL'], stock_result['PREDICTED']) * 100
# Calculate MPE % 
mpe = mean_positive_error(stock_result['ACTUAL'],stock_result['PREDICTED'])
# Calculate MTT in seconds 
mtt = np.mean(stock_result['TRAIN_DURATION'])
new_row = pd.Series(['F',rmse, mape, mpe, mtt], index=metrics_df.columns)
new_rows.append(new_row)
metrics_df = pd.DataFrame(new_rows)
metrics_df

,stock_ID,RMSE,MAPE,MPE,MTT
0,F,0.790372,4.847643,0.37914,0.36411


In [21]:
new_rows = []
stock_result = nvda_result
# Calculate RMSE 
rmse = np.sqrt(mean_squared_error(stock_result['ACTUAL'], stock_result['PREDICTED']))
# Calculate MAPE % 
mape = mean_absolute_percentage_error(stock_result['ACTUAL'], stock_result['PREDICTED']) * 100
# Calculate MPE % 
mpe = mean_positive_error(stock_result['ACTUAL'],stock_result['PREDICTED'])
# Calculate MTT in seconds 
mtt = np.mean(stock_result['TRAIN_DURATION'])
new_row = pd.Series(['NVDA',rmse, mape, mpe, mtt], index=metrics_df.columns)
new_rows.append(new_row)
metrics_df = pd.DataFrame(new_rows)
metrics_df

,stock_ID,RMSE,MAPE,MPE,MTT
0,NVDA,9.591114,5.452567,2.99903,0.401929


In [22]:
new_rows = []
stock_result = tsla_result
# Calculate RMSE 
rmse = np.sqrt(mean_squared_error(stock_result['ACTUAL'], stock_result['PREDICTED']))
# Calculate MAPE % 
mape = mean_absolute_percentage_error(stock_result['ACTUAL'], stock_result['PREDICTED']) * 100
# Calculate MPE % 
mpe = mean_positive_error(stock_result['ACTUAL'],stock_result['PREDICTED'])
# Calculate MTT in seconds 
mtt = np.mean(stock_result['TRAIN_DURATION'])
new_row = pd.Series(['TSLA',rmse, mape, mpe, mtt], index=metrics_df.columns)
new_rows.append(new_row)
metrics_df = pd.DataFrame(new_rows)
metrics_df

,stock_ID,RMSE,MAPE,MPE,MTT
0,TSLA,23.907778,9.936937,13.759807,0.422963


In [23]:
new_rows = []
stock_result = wmt_result
# Calculate RMSE 
rmse = np.sqrt(mean_squared_error(stock_result['ACTUAL'], stock_result['PREDICTED']))
# Calculate MAPE % 
mape = mean_absolute_percentage_error(stock_result['ACTUAL'], stock_result['PREDICTED']) * 100
# Calculate MPE % 
mpe = mean_positive_error(stock_result['ACTUAL'],stock_result['PREDICTED'])
# Calculate MTT in seconds 
mtt = np.mean(stock_result['TRAIN_DURATION'])
new_row = pd.Series(['WMT',rmse, mape, mpe, mtt], index=metrics_df.columns)
new_rows.append(new_row)
metrics_df = pd.DataFrame(new_rows)
metrics_df

,stock_ID,RMSE,MAPE,MPE,MTT
0,WMT,4.581791,2.61167,1.038705,0.398609
